In [1]:
import numpy
import scipy.special
import matplotlib.pyplot

In [2]:
class neuralNetwork:
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        self.inodes = input_nodes
        self.hnodes = hidden_nodes
        self.onodes = output_nodes
        
        self.lr = learning_rate
        
        self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))
        
        self.activation_function = lambda x: scipy.special.expit(x)
        
        
    
    def train(self, inputs_list, targets_list):
        inputs = numpy.array(inputs_list, ndmin = 2).T
        
        hidden_inputs = numpy.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        output_inputs = numpy.dot(self.who, hidden_outputs)
        output_finals = self.activation_function(output_inputs)
        targets = numpy.array(targets_list, ndmin = 2).T
        
        #calculating main output errors
        #here starts error propagation algorithm
        output_errors = targets - output_finals
        hidden_errors = numpy.dot(self.who.T, output_errors)
        
        #correcting weight coefficients
        self.who += self.lr * numpy.dot(output_errors * output_finals * (1 - output_finals), 
                                        numpy.transpose(hidden_outputs))
        self.wih += self.lr * numpy.dot(hidden_errors * hidden_outputs * (1 - hidden_outputs),
                                        numpy.transpose(inputs))
        
        
    
    def query(self, inputs_list):
        #transforming incoming list to 2d array
        inputs = numpy.array(inputs_list, ndmin = 2).T
        
        hidden_inputs = numpy.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        output_inputs = numpy.dot(self.who, hidden_outputs)
        output_finals = self.activation_function(output_inputs)
        
        return output_finals

In [15]:
#initialising NN

input_nodes = 784 #we have 28 * 28 = 784 image
hidden_nodes = 200 # we choose it by ourselves
output_nodes = 10 #we need to classify ten digits
learning_rate = 0.02
epochs = 8
    
nn = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [16]:
#uploading short training dataset

train_data_file = open("mnist_datasets/mnist_train.csv", 'r')
train_data_list = train_data_file.readlines()
train_data_file.close()

In [17]:
#training NN using training dataset
for e in range(epochs):
    for digit in train_data_list:
        vals = digit.split(',')

        #transforming input data to correct data type(all valuse should be form 0.01 to 1)
        correct_inputs = (numpy.asfarray(vals[1:]) / 255.0 * 0.99) + 0.01

        #correcting targets
        targets = numpy.zeros(output_nodes) + 0.01
        targets[int(vals[0])] = 0.99
    
        nn.train(correct_inputs, targets)

In [18]:
#uploading test dataset

test_data_file = open("mnist_datasets/mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

#here would be located right or wrong answers - 1 or 0
scores = []

In [19]:
for digit in test_data_list:
    test_vals = digit.split(',')
    correct_label = int(test_vals[0])
    
    
    correct_inputs = (numpy.asfarray(test_vals[1:]) / 255.0 * 0.99) + 0.01
    outputs = nn.query(correct_inputs)
    
    nn_label = numpy.argmax(outputs)
    
    if(nn_label == correct_label):
        scores.append(1)
    else:
        scores.append(0)
    

In [20]:
scores_array = numpy.asarray(scores)
efficiency = scores_array.sum() / scores_array.size
print("NN Efficiency: ", efficiency)

NN Efficiency:  0.9736
